In [8]:
pip install ultralytics

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflowlite (from versions: none)
ERROR: No matching distribution found for tensorflowlite


In [3]:
pip install supervision

Note: you may need to restart the kernel to use updated packages.


In [23]:
#simple code withoutt no rules 

import cv2
from ultralytics import YOLO
import numpy as np

def run_realtime_detection():
    # Load YOLO model
    print("Loading YOLO model...")
    model = YOLO('yolov8n.pt')
    
    # Initialize webcam
    print("Starting webcam...")
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("Error: Couldn't open webcam")
        return
    
    print("Detection starting... Press 'q' to quit")
    
    try:
        while True:
            # Read frame
            ret, frame = cap.read()
            if not ret:
                print("Error: Couldn't read frame")
                break
            
            # Run detection
            results = model(frame, stream=True)
            
            # Process results
            for result in results:
                # Get the boxes and confidence scores
                boxes = result.boxes.cpu().numpy()
                
                # Draw boxes
                for box in boxes:
                    # Get box coordinates
                    x1, y1, x2, y2 = box.xyxy[0].astype(int)
                    
                    # Get class name and confidence
                    class_id = int(box.cls[0])
                    conf = box.conf[0]
                    class_name = model.names[class_id]
                    
                    # Draw rectangle
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    
                    # Add label
                    label = f'{class_name} {conf:.2f}'
                    cv2.putText(frame, label, (x1, y1-10), 
                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            
            # Show frame
            cv2.imshow('Real-time Object Detection', frame)
            
            # Break loop with 'q'
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
    finally:
        # Clean up
        cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    run_realtime_detection()

Loading YOLO model...
Starting webcam...
Detection starting... Press 'q' to quit

0: 480x640 3 persons, 45.0ms
Speed: 1.0ms preprocess, 45.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 26.6ms
Speed: 1.0ms preprocess, 26.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 33.3ms
Speed: 1.4ms preprocess, 33.3ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 31.5ms
Speed: 0.0ms preprocess, 31.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 30.4ms
Speed: 1.0ms preprocess, 30.4ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 31.6ms
Speed: 0.0ms preprocess, 31.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 29.3ms
Speed: 0.0ms preprocess, 29.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 28.2ms


In [27]:
import cv2
from ultralytics import YOLO
import numpy as np
from collections import defaultdict
import time

class ObjectDetectionTracker:
    def __init__(self, model_path='yolov8n.pt', confidence_threshold=0.5, iou_threshold=0.3):
        """
        Initialize the object detection and tracking system
        """
        self.model = YOLO(model_path)
        self.conf_threshold = confidence_threshold
        self.iou_threshold = iou_threshold
        self.tracking_objects = {}
        self.track_id = 0
        self.fps_history = []
        
    def calculate_iou(self, box1, box2):
        """Calculate Intersection over Union between two boxes"""
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])
        
        intersection = max(0, x2 - x1) * max(0, y2 - y1)
        area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
        area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
        union = area1 + area2 - intersection
        
        return intersection / union if union > 0 else 0

    def process_frame(self, frame):
        """
        Process a single frame for object detection and tracking
        """
        # Store original frame dimensions
        height, width = frame.shape[:2]
        
        # Detect objects
        results = self.model(frame, stream=True)
        
        # Process detection results
        current_objects = {}
        
        for result in results:
            boxes = result.boxes.cpu().numpy()
            
            for box in boxes:
                # Get detection information
                x1, y1, x2, y2 = box.xyxy[0].astype(int)
                conf = box.conf[0]
                class_id = int(box.cls[0])
                class_name = self.model.names[class_id]
                
                if conf < self.conf_threshold:
                    continue
                    
                # Check if object overlaps with any existing tracked object
                matched = False
                for track_id, tracked_obj in self.tracking_objects.items():
                    tracked_box = tracked_obj['box']
                    iou = self.calculate_iou([x1, y1, x2, y2], tracked_box)
                    
                    if iou > self.iou_threshold:
                        current_objects[track_id] = {
                            'box': [x1, y1, x2, y2],
                            'class': class_name,
                            'confidence': conf,
                            'tracked_frames': tracked_obj['tracked_frames'] + 1
                        }
                        matched = True
                        break
                
                # If no match found, create new tracked object
                if not matched:
                    self.track_id += 1
                    current_objects[self.track_id] = {
                        'box': [x1, y1, x2, y2],
                        'class': class_name,
                        'confidence': conf,
                        'tracked_frames': 1
                    }
        
        # Update tracking objects
        self.tracking_objects = current_objects
        
        return self.tracking_objects

    def draw_results(self, frame, objects):
        """
        Draw detection and tracking results on frame
        """
        for track_id, obj in objects.items():
            x1, y1, x2, y2 = obj['box']
            class_name = obj['class']
            conf = obj['confidence']
            tracked_frames = obj['tracked_frames']
            
            # Draw box with different colors based on tracking duration
            color = (0, int(min(255, tracked_frames * 10)), 0)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            
            # Draw label with tracking ID
            label = f"#{track_id} {class_name} {conf:.2f}"
            cv2.putText(frame, label, (x1, y1-10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            
        return frame

    def run_detection(self, source=0, display=True):
        """
        Run real-time detection and tracking
        """
        cap = cv2.VideoCapture(source)
        
        if not cap.isOpened():
            print("Error: Couldn't open video source")
            return
        
        print("Detection and tracking started... Press 'q' to quit")
        
        try:
            while True:
                start_time = time.time()
                
                ret, frame = cap.read()
                if not ret:
                    break
                
                # Process frame
                tracked_objects = self.process_frame(frame)
                
                # Draw results
                if display:
                    frame = self.draw_results(frame, tracked_objects)
                    
                    # Calculate and display FPS
                    fps = 1.0 / (time.time() - start_time)
                    self.fps_history.append(fps)
                    if len(self.fps_history) > 30:
                        self.fps_history.pop(0)
                    avg_fps = sum(self.fps_history) / len(self.fps_history)
                    
                    cv2.putText(frame, f'FPS: {avg_fps:.1f}', (10, 30),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    
                    cv2.imshow('Object Detection and Tracking', frame)
                
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                    
        finally:
            cap.release()
            if display:
                cv2.destroyAllWindows()

def main():
    # Initialize detector with custom settings
    detector = ObjectDetectionTracker(
        model_path='yolov8n.pt',
        confidence_threshold=0.5,
        iou_threshold=0.3
    )
    
    # Run detection
    detector.run_detection()

if __name__ == "__main__":
    main()

Detection and tracking started... Press 'q' to quit

0: 480x640 2 persons, 52.2ms
Speed: 2.0ms preprocess, 52.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 29.5ms
Speed: 1.0ms preprocess, 29.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 30.6ms
Speed: 0.0ms preprocess, 30.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 26.8ms
Speed: 0.0ms preprocess, 26.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 31.7ms
Speed: 1.0ms preprocess, 31.7ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 30.0ms
Speed: 0.0ms preprocess, 30.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 30.0ms
Speed: 0.0ms preprocess, 30.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 30.3ms
Speed: 1.0ms preprocess, 30.3